# NLP COVID Vax Cleaning

In [1]:
import pandas as pd
import numpy as np

import re
import string

pd.set_option("max_colwidth", 0)
pd.set_option('max_columns', 35)
np.set_printoptions(suppress=True) # Suppress scientific notation where possible

## Explore & clean data 

In [2]:
# import dataset from prior notebook
# we have already removed duplicates (based on 'text' column) and non-english

vax_tweets = pd.read_csv('vax_tweets.csv')
vax_tweets.head()

/Users/mayaremington/opt/anaconda3/envs/metis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,6,22,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Sun Nov 01 07:04:54 +0000 2020,NaN,NaN,https://twitter.com/LotusOak2/status/1322642287449591814,0,1322796743298265088,NaN,NaN,NaN,en,NaN,False,1.322642e+18,0.0,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",Stay away from that vaccine. Population control. https://t.co/DTMFGsNRzX,https://twitter.com/BAChatwin/status/1322796743298265088,Mon Sep 19 17:13:57 +0000 2011,3.763047e+08,False,☀️🇨🇦🌍Truth☀️🇨🇦🌍 Freedom ☀️🇨🇦🌏Health ☀️🇨🇦🌍Eyes wide open\nAwake patriot and CF veteran (13+yrs),1643.0,130.0,658.0,0.0,"Niagara, Ontario",Barb Chatwin,BAChatwin,2659.0,NaN,NaN,False
1,NaN,Sun Nov 01 07:10:02 +0000 2020,NaN,NaN,NaN,0,1322798033596272640,NaN,NaN,NaN,en,NaN,NaN,NaN,489.0,1.322434e+18,sunnynwaobi1,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>","Isn't this a systematic depopulation happing in Obigbo?, where young unarmed citizens are randomly picked, some tortured while some are killed. #Obigbomassacre @StateDept @AmnestyNigeria @ https://t.co/TsakU0l3Fm",https://twitter.com/jumoke_ladan/status/1322798033596272640,Wed Jul 01 19:30:15 +0000 2009,5.282430e+07,False,Am a Christian by God's grace. Called by Christ to serve and love Him as a Catholic. Perpetually loving Him.,7009.0,224.0,359.0,1.0,"Lagos, Nigeria",Olajumoke Ladan,jumoke_ladan,1812.0,NaN,NaN,False
2,NaN,Sun Nov 01 07:13:25 +0000 2020,NaN,NaN,NaN,0,1322798887694905346,NaN,NaN,NaN,en,NaN,NaN,NaN,122.0,1.322792e+18,conspiracyb0t,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",Bill Gates is one of the biggest proponents of mass depopulation.,https://twitter.com/wambo161/status/1322798887694905346,Fri Apr 08 16:05:41 +0000 2011,2.791067e+08,False,Follower of Jesus.,27413.0,205.0,711.0,3.0,la montaña de roble,James of Oak Mountain,wambo161,48229.0,NaN,NaN,False
3,NaN,Sun Nov 01 07:31:24 +0000 2020,NaN,NaN,NaN,0,1322803412300431361,NaN,NaN,NaN,en,NaN,NaN,1.322434e+18,41.0,1.322774e+18,Onyinye51838197,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",The world should be on notice that Nelson nwike river state governor and the Nigeria army are committing genocide and pogrom in Biafra land https://t.co/fTC8gNNSsZ,https://twitter.com/EmmanuelNwanyim/status/1322803412300431361,Tue Apr 21 07:16:59 +0000 2020,1.252496e+18,False,i was born to shine take it or live it...blessed son by God in heaven..🙌🏼🙌🏼🙌🏼..peace 👍🏻 ...Biafran citizen Israel 🇮🇱 citizen,8947.0,121.0,307.0,1.0,Israel,Emmanuel nwanyim,EmmanuelNwanyim,16345.0,NaN,NaN,False
4,NaN,Sun Nov 01 07:01:51 +0000 2020,NaN,NaN,NaN,1,1322795973337403395,bongzmessi,1.322170e+18,217711495.0,en,NaN,NaN,NaN,0.0,NaN,NaN,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",@bongzmessi To come to think of it perhaps this was done so they multiply and outnumber us but. To them the depopulation agenda is at top of their list. Phela my King bake baphahluke labantu telling things they were not supposed like the one on an interview that they injected AIDS to blacks,https://twitter.com/Boniswa63207956/status/1322795973337403395,Mon May 25 12:43:06 +0000 2020,1.264900e+18,False,Mayibuye iAfrika✊🏾. \n\nI don't call my people Kings and Queens to tickle my throat it's because we are Royalty 👸🏾🤴🏿.\nTribalism 🤮Feminism 🤮Xenophobia🤮,39392.0,3658.0,3680.0,2.0,NaN,Boniswa,Boniswa63207956,40107.0,NaN,NaN,False


In [3]:
vax_tweets.shape

(396609, 35)

In [4]:
vax_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396609 entries, 0 to 396608
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   coordinates                 219 non-null     object 
 1   created_at                  396608 non-null  object 
 2   hashtags                    64258 non-null   object 
 3   media                       31453 non-null   object 
 4   urls                        141888 non-null  object 
 5   favorite_count              396607 non-null  object 
 6   id                          396608 non-null  object 
 7   in_reply_to_screen_name     208688 non-null  object 
 8   in_reply_to_status_id       203126 non-null  float64
 9   in_reply_to_user_id         208686 non-null  float64
 10  lang                        396606 non-null  object 
 11  place                       7499 non-null    object 
 12  possibly_sensitive          165879 non-null  object 
 13  quote_id      

### Basic cleaning 

In [5]:
vax_tweets['text'].nunique()

396605

In [6]:
# seems there are 4 rows with nulls in text column

vax_tweets[vax_tweets['text'].isnull()]

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
169727,#SwissCovidFail https://t.co/Oq5Wo5Q9iI,https://twitter.com/GrrrrAngryShark/status/1368990271233527809,Fri Jun 19 17:44:27 +0000 2015,3335258632,False,Get vaxxed or get the Hell out of my face. I'm done playing. \n\nhttps://t.co/n9DARXT3Nv\n\n#TeamFangirls (She/her),98266,1240,2065.0,124.0,"Hell, apparently","Grrrr, the Angry Shark",GrrrrAngryShark,245703.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304865,You continue to read this too.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304866,Now you will stop at the end of this line.,12507,590,275,0,NaN,Hassan,RaiyDhidha,4037.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368816,Always on to the next thing.,389,318,209,0,RANCHI,JAYANT GHOSH,JAYANTMAATI,2085.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# dropping the 4 nulls

vax_tweets.dropna(subset=['text'], inplace=True)

In [8]:
vax_tweets.shape

(396605, 35)

In [9]:
# convert id's to integers (in order to suppress scientific notation)

vax_tweets['quote_id'] = vax_tweets['quote_id'].astype('Int64')
vax_tweets['retweet_id'] = vax_tweets['retweet_id'].astype('Int64')
vax_tweets['user_id'] = vax_tweets['user_id'].astype('Int64')

In [10]:
# convert 'created_at' to DataTime

vax_tweets['created_at'] = vax_tweets['created_at'].apply(lambda x: pd.to_datetime(x))

### Selecting most useful columns 

In [11]:
# these are the only columns we need -> new, smaller DF

vax_df = vax_tweets.loc[:, ['created_at', 'hashtags', 'in_reply_to_screen_name',
                  'quote_id', 'retweet_id', 'retweet_screen_name', 
                  'user_id', 'user_screen_name','user_description',
                   'user_location', 'text']]
vax_df.head()

,created_at,hashtags,in_reply_to_screen_name,quote_id,retweet_id,retweet_screen_name,user_id,user_screen_name,user_description,user_location,text
0,2020-11-01 07:04:54+00:00,NaN,NaN,1322642287449591808,<NA>,NaN,376304718,BAChatwin,☀️🇨🇦🌍Truth☀️🇨🇦🌍 Freedom ☀️🇨🇦🌏Health ☀️🇨🇦🌍Eyes wide open\nAwake patriot and CF veteran (13+yrs),"Niagara, Ontario",Stay away from that vaccine. Population control. https://t.co/DTMFGsNRzX
1,2020-11-01 07:10:02+00:00,NaN,NaN,<NA>,1322434241829982208,sunnynwaobi1,52824297,jumoke_ladan,Am a Christian by God's grace. Called by Christ to serve and love Him as a Catholic. Perpetually loving Him.,"Lagos, Nigeria","Isn't this a systematic depopulation happing in Obigbo?, where young unarmed citizens are randomly picked, some tortured while some are killed. #Obigbomassacre @StateDept @AmnestyNigeria @ https://t.co/TsakU0l3Fm"
2,2020-11-01 07:13:25+00:00,NaN,NaN,<NA>,1322791741854937088,conspiracyb0t,279106686,wambo161,Follower of Jesus.,la montaña de roble,Bill Gates is one of the biggest proponents of mass depopulation.
3,2020-11-01 07:31:24+00:00,NaN,NaN,1322434241829982208,1322774415873052672,Onyinye51838197,1252496422865375232,EmmanuelNwanyim,i was born to shine take it or live it...blessed son by God in heaven..🙌🏼🙌🏼🙌🏼..peace 👍🏻 ...Biafran citizen Israel 🇮🇱 citizen,Israel,The world should be on notice that Nelson nwike river state governor and the Nigeria army are committing genocide and pogrom in Biafra land https://t.co/fTC8gNNSsZ
4,2020-11-01 07:01:51+00:00,NaN,bongzmessi,<NA>,<NA>,NaN,1264899714950062080,Boniswa63207956,Mayibuye iAfrika✊🏾. \n\nI don't call my people Kings and Queens to tickle my throat it's because we are Royalty 👸🏾🤴🏿.\nTribalism 🤮Feminism 🤮Xenophobia🤮,NaN,@bongzmessi To come to think of it perhaps this was done so they multiply and outnumber us but. To them the depopulation agenda is at top of their list. Phela my King bake baphahluke labantu telling things they were not supposed like the one on an interview that they injected AIDS to blacks


In [12]:
vax_df.shape

(396605, 11)

In [13]:
# 12K of the 396K unique tweets are re-tweets

(~(vax_df['retweet_id'].isnull())).sum()

12460

In [14]:
# 99K of the 396K unique tweets are quote-tweets

(~(vax_df['quote_id'].isnull())).sum()

99011

In [15]:
# top hashtags (although this is unvectorized data - so each combination of # is it's own category)

vax_df['hashtags'].value_counts().head(30)

NoVaccineForMe              1812
MyBodyMyChoice              1509
mybodymychoice              1402
InformedConsent             1048
NoVaccine                   964 
BillGatesBioTerrorist       815 
COVID19                     658 
IDoNotConsent               523 
depopulation                416 
SackSage ArrestBillGates    302 
vaxxed                      291 
informedconsent             246 
VAXXED                      243 
Depopulation                196 
novaccine                   184 
ArrestBillGates             159 
ExposeBillGates             157 
vaccine                     144 
VaccinePassports            142 
CovidVaccine                141 
LearnTheRisk                135 
arrestbillgates             132 
AntiVaccine                 128 
DemVoice1                   124 
Vaxxed                      122 
BigPharma VaccineInjury     119 
VaccineInjury               113 
Enjoy RussianRoulette       109 
Idonotconsent               108 
NoVaccinePassports          103 
Name: hash

### Identify text elements that will need preprocessing 

In [16]:
# find all mentions

def find_mentions(tweet):
    return re.findall('(@[A-Za-z]+[A-Za-z0-9-_]+)', tweet)

In [17]:
vax_df['text'].apply(find_mentions)

0         []                           
1         [@StateDept, @AmnestyNigeria]
2         []                           
3         []                           
4         [@bongzmessi]                
              ...                      
396604    [@eileeniorio, @pfizer]      
396605    [@SBSNews]                   
396606    []                           
396607    []                           
396608    []                           
Name: text, Length: 396605, dtype: object

In [18]:
def find_hashtags(tweet):
    return re.findall('(#[A-Za-z]+[A-Za-z0-9-_]+)', tweet)

In [19]:
vax_df['text'].apply(find_hashtags)

0         []               
1         [#Obigbomassacre]
2         []               
3         []               
4         []               
          ..               
396604    []               
396605    []               
396606    []               
396607    []               
396608    []               
Name: text, Length: 396605, dtype: object

In [20]:
def find_http_links(tweet):
    return re.findall(r'http\S+', tweet)

In [21]:
vax_df['text'].apply(find_http_links)

0         [https://t.co/DTMFGsNRzX]
1         [https://t.co/TsakU0l3Fm]
2         []                       
3         [https://t.co/fTC8gNNSsZ]
4         []                       
          ..                       
396604    []                       
396605    []                       
396606    [https://t.co/SFXefBOM9A]
396607    []                       
396608    []                       
Name: text, Length: 396605, dtype: object

In [22]:
def find_bitly_links(tweet):
    return re.findall(r'bit.ly/\S+', tweet)

In [23]:
# there's one of this link type

vax_df['text'].apply(find_bitly_links).sum()

['bit.ly/3v79D3x']

## Export df 

In [24]:
# export for use in next notebook

vax_df.to_csv('vax_df.csv', index=False)

In [25]:
vax_df.shape

(396605, 11)